In [3]:
import numpy as np
import random
qubit_0 =  np.array([1,0]).reshape(2,)
qubit_1 =  np.array([0,1]).reshape(2,)
qubit_000 = np.kron(np.kron(qubit_0,qubit_0),qubit_0)
qubit_111 = np.kron(np.kron(qubit_1,qubit_1),qubit_1)
c = random.random()
c




0.9712576547423558

In [4]:
qubit_final=(qubit_000 + c* qubit_111) * np.sqrt((1/(1+c**2)))
qubit_final

array([0.71734066, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.6967226 ])

In [2]:
import numpy as np
import random 
CNOT_matrix=np.array([[1,0,0,0],
                      [0,1,0,0],
                      [0,0,0,1],
                      [0,0,1,0]])
hadamard=(1/(np.sqrt(2)))*np.array([[1, 1],[1,-1]])

swap= np.array([[1,0,0,0],
                [0,0,1,0],
                [0,1,0,0],
                [0,0,0,1]])

qubit_0 =  np.array([1,0]).reshape(2,)
qubit_1 =  np.array([0,1]).reshape(2,)
qubit_00 = np.kron(qubit_0,qubit_0)
qubit_11 = np.kron(qubit_1,qubit_1)
c = random.random()
c




0.33642526038329723

In [5]:
identity = np.identity(2)

#prepration of 4 qubit state |1010>
psi_1 = ((qubit_00 + c* qubit_11) * np.sqrt((1/(1+c**2)))).reshape(4,)
psi_2 = ((qubit_00 + c* qubit_11) * np.sqrt((1/(1+c**2)))).reshape(4,)
psi = np.kron(psi_1,psi_2)

#performing swap operation |1010> -> |1100>
swap_tensored =np.kron(identity,np.kron(swap,identity))
swaped_state= np.matmul(swap_tensored,psi)


#cnot operation on swapped qubits |1100> -> |1000>
cnot = np.kron(CNOT_matrix,CNOT_matrix)
state_after_cnot = np.matmul(cnot,swaped_state)

#swapping the states back |1000> -> |1000>
original_psi = np.matmul(swap_tensored,state_after_cnot)

#hadamard operation |1000> -> |0000> + |0100> - |1000> - |1100>
tensored_hadamard = np.kron(hadamard,np.kron(hadamard,np.identity(4)))
final_state_after_hadamard =np.matmul(tensored_hadamard,original_psi)


#measurement
projector=np.array([[1,0],[0,0]])
projector_tensored = np.kron(np.identity(4),np.kron(projector,projector))

probablity = np.dot(final_state_after_hadamard.reshape(1,16),np.dot(projector_tensored,final_state_after_hadamard))


#tracing out the measured copy
first_copy_of_the_state = np.trace(final_state_after_hadamard.reshape(2,2,2,2),axis1=2,axis2=3)


#creating bell state
bell_state = (np.array([[1],[0],[0],[1]]) * (1/ np.sqrt(2))).reshape(4,)


# preparation of density matrix of bell state and psi state by taking the outer product between the states
density_matrix_bell = np.outer(bell_state,bell_state) 

#dividing by the trace of density matrix to make it 1
density_matrix_pp = np.outer(first_copy_of_the_state,first_copy_of_the_state) / np.trace(np.outer(first_copy_of_the_state,first_copy_of_the_state))


#calculation of fidelity
x = np.matmul(bell_state.reshape(1,4),np.matmul(density_matrix_pp,bell_state))
fidelity = np.sqrt(x)



final_state_after_hadamard , first_copy_of_the_state, probablity , fidelity, 






(array([5.00000000e-01, 0.00000000e+00, 0.00000000e+00, 3.02219470e-01,
        3.98325736e-01, 0.00000000e+00, 0.00000000e+00, 5.79752576e-18,
        3.98325736e-01, 0.00000000e+00, 0.00000000e+00, 5.79752576e-18,
        5.00000000e-01, 0.00000000e+00, 0.00000000e+00, 3.02219470e-01]),
 array([[0.80221947, 0.39832574],
        [0.39832574, 0.80221947]]),
 array([0.81732678]),
 array([0.89566705]))